In [446]:
import pandas as pd
import numpy as np
import joblib
pd.options.mode.chained_assignment = None

In [447]:
train_trans = pd.read_csv('data/train_trans.csv')

In [819]:
train_base = pd.read_csv('data/train_base.csv')
test_a_base = pd.read_csv('data/test_a_base.csv')

train_op = pd.read_csv('data/train_op.csv')
test_a_op = pd.read_csv('data/test_a_op.csv')

train_trans = pd.read_csv('data/train_trans.csv')
test_a_trans = pd.read_csv('data/test_a_trans.csv')

train_label = pd.read_csv('data/train_label.csv')

In [820]:
train_base = pd.merge(train_base, train_label, on='user', how='left')

## base处理

In [821]:
train_base_no_nan = train_base.drop(columns=['service3_level'],axis=1)
# 离散值填充众数
train_base_no_nan['sex'].fillna('category 0',inplace=True)
train_base_no_nan['balance_avg'].fillna('level 1',inplace=True)
train_base_no_nan['balance1_avg'].fillna('level 1',inplace=True)
train_base_no_nan['balance2_avg'].fillna('level 1',inplace=True)

In [761]:
user = train_base_no_nan['user'].values
train_base_no_user = train_base_no_nan.drop(columns=['user'],axis=1)

In [762]:
# 少的做one-hot
from sklearn.preprocessing import OneHotEncoder

category_columns_name = ['sex','provider','level','verified','regist_type','agreement1','agreement2','agreement3','agreement4']
# ,'city','province'
ohe = OneHotEncoder()
ohe_encoder = ohe.fit(train_base_no_user[category_columns_name])
one_hot_value = ohe_encoder.transform(train_base_no_user[category_columns_name]).toarray()

one_hot_df = pd.DataFrame(one_hot_value)
joblib.dump(ohe_encoder,'ohe_encoder.pkl')

In [764]:
train_base_categoty = train_base_no_user.select_dtypes('O')

for c in list(train_base_categoty.columns[0:9])+['service3']:
    train_base_categoty[c]=train_base_categoty[c].apply(lambda x:int(x.strip().split(' ')[1]))
    
for c in list(train_base_categoty.columns[11:17])+list(train_base_categoty.columns[18:25]):
    train_base_categoty[c]=train_base_categoty[c].apply(lambda x:int(x.strip().split(' ')[1]))

In [814]:
cumsu = train_base_no_user.groupby('city')['label'].cumsum() - train_base_no_user['label']
cumcnt = train_base_no_user.groupby('city').cumcount()+1e-10
train_base_categoty['city'] = cumsu/cumcnt

cumsu = train_base_no_user.groupby('province')['label'].cumsum() - train_base_no_user['label']
cumcnt = train_base_no_user.groupby('province').cumcount()+1e-10
train_base_categoty['province'] = cumsu/cumcnt

In [815]:
train_base_categoty.groupby('user')['city'].mean()

KeyError: 'user'

In [766]:
# city_dic = dict(train_base_no_user.groupby('city')['label'].sum()/train_base_no_user.groupby('city')['label'].count())
# province_dic = dict(train_base_no_user.groupby('province')['label'].sum()/train_base_no_user.groupby('province')['label'].count())

# train_base_categoty['city'] = train_base_categoty['city'].apply(lambda x:city_dic[x])
# train_base_categoty['province'] = train_base_categoty['province'].apply(lambda x:province_dic[x])

In [767]:
joblib.dump(city_dic,'city_dic.pkl')
joblib.dump(province_dic,'province_dic.pkl')

['province_dic.pkl']

In [768]:
# train_base_categoty.drop(columns=['province','city'],axis=1,inplace=True)
train_base_categoty.drop(columns=category_columns_name,axis=1,inplace=True)

In [769]:
train_base_categoty

,province,city,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount
0,0.000000,0.000000,1,1,1,1,1,1,0,1,1,1,0,0,1
1,0.000000,0.000000,19,19,1,1,20,5,0,1,1,1,0,0,1
2,0.000000,0.000000,7,5,14,11,1,5,0,1,1,1,0,0,1
3,0.000000,0.000000,11,7,1,1,12,4,0,1,1,1,0,0,1
4,0.000000,0.000000,8,9,1,1,9,5,0,2,8,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47777,0.289634,0.287368,1,9,1,1,1,4,0,1,1,1,0,0,1
47778,0.209649,0.187879,9,10,1,1,10,5,0,1,1,1,0,0,1
47779,0.312567,0.304348,1,1,1,1,1,1,0,1,1,1,0,0,1
47780,0.227984,0.193906,2,13,2,3,2,5,0,2,2,1,0,0,9


In [770]:
# joblib.dump(ohe_encoder,'one_encoder.pkl')

In [771]:
number_value = train_base_no_user.select_dtypes('int64').values
df_value= pd.DataFrame(number_value)
df_value.columns = train_base_no_user.select_dtypes('int64').columns

In [772]:
df_value['product7_success_cnt'] = df_value['product7_cnt']-df_value['product7_fail_cnt']
df_value['card_cnt'] = df_value['card_a_cnt']+df_value['card_b_cnt']+df_value['card_c_cnt']+df_value['card_d_cnt']
df_value['ip_cnt_avg'] = df_value['ip_cnt']/df_value['login_days_cnt']
df_value['login_cnt_period']=df_value['login_cnt_period1']+df_value['login_cnt_period2']
df_value['login_cnt_period1_avg']=df_value['login_cnt_period1']/df_value['login_days_cnt']
df_value['login_cnt_period2_avg']=df_value['login_cnt_period2']/df_value['login_days_cnt']
df_value['login_cnt_period_avg']=df_value['login_cnt_period']/df_value['login_days_cnt']
df_value['using_time_avg'] = df_value['using_time']/df_value['login_days_cnt']
df_value['product7_success_cnt_avg'] = df_value['product7_success_cnt']/df_value['login_days_cnt']
df_value['product7_cnt_avg']=df_value['product7_cnt']/df_value['login_days_cnt']
df_value['product7_fail_cnt_avg']=df_value['product7_fail_cnt']/df_value['login_days_cnt']
df_value['service_cnt']=df_value['service1_cnt']+df_value['service2_cnt']
df_value['service_cnt_avg']= df_value['service_cnt']/df_value['login_days_cnt']
df_value['service_amt_cnt_avg']=df_value['service1_amt']/df_value['service_cnt']

In [773]:
# 归一化
df_value = (df_value-df_value.min())/(df_value.max()-df_value.min())
# df_value = (df_value-df_value.mean())/df_value.std()

In [774]:
# df_one_hot = pd.DataFrame(one_hot_value)

df_base = pd.DataFrame()
df_base['user'] = user

df_base = pd.concat([df_base, train_base_categoty,df_value,one_hot_df],axis=1)

In [775]:
df_base.shape

(47782, 77)

In [779]:
df_base.head()

,user,province,city,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,label,product7_success_cnt,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period1_avg,login_cnt_period2_avg,login_cnt_period_avg,using_time_avg,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,Train_06800,0.0,0.0,1,1,1,1,1,1,0,1,1,1,0,0,1,0.235862,0.415094,0.057692,0.098361,0.000000,0.091346,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.004402,0.003142,0.010150,0.021058,0.078467,0.00000,0.0,0.0,0.000000,0.057692,0.225496,0.003810,0.004135,0.002801,0.003509,0.883054,0.000000,0.834355,0.834355,0.0,0.899806,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,Train_23487,0.0,0.0,19,19,1,1,20,5,0,1,1,1,0,0,1,0.260690,0.245283,0.125000,0.213115,0.000000,0.062500,0.027397,0.0,0.252033,0.0,0.0,0.0,0.001736,0.004759,0.004061,0.045406,0.012243,0.167883,0.04878,0.0,1.0,0.857143,0.125000,0.235239,0.004431,0.004076,0.003265,0.003695,0.783694,0.853963,0.762022,0.751909,0.0,0.810893,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,Train_36880,0.0,0.0,7,5,14,11,1,5,0,1,1,1,0,0,1,0.202759,0.320755,0.125000,0.213115,0.000000,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.005498,0.019068,0.010166,0.123932,0.018119,0.322993,0.04878,0.0,1.0,0.857143,0.125000,0.270565,0.014887,0.017896,0.008655,0.013557,0.645769,0.851046,0.619739,0.609661,0.0,0.657486,0.000008,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,Train_35392,0.0,0.0,11,7,1,1,12,4,0,1,1,1,0,0,1,0.320000,0.018868,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.203252,0.0,0.0,0.0,0.005498,0.003347,0.003779,0.049145,0.012243,0.155109,0.04878,0.0,0.0,0.857143,0.057692,0.243241,0.003550,0.002696,0.003046,0.002860,0.775081,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,Train_35057,0.0,0.0,8,9,1,1,9,5,0,2,8,1,0,0,1,0.344828,0.415094,0.125000,0.098361,0.070588,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.001736,0.003646,0.001435,0.032585,0.002938,0.155109,0.04878,0.0,1.0,0.857143,0.120192,0.225519,0.002607,0.003001,0.000651,0.001898,0.811092,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


## OP处理

In [741]:
train_op.head()

,user,op_type,op_mode,op_device,ip,net_type,channel,ip_3,tm_diff
0,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:22.000000000
1,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:21.000000000
2,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:23.000000000
3,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:26.000000000
4,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:41.000000000


In [742]:
op_df = pd.DataFrame()
group = train_op.groupby(['user']).count()
op_df['user'] = group.index

In [743]:
count_column=train_op.columns[1:-1]
for column in count_column:
    print(column)
#     des_df = train_op.groupby('user')[column].describe()
#     op_df['op_'+column+'_count'] = des_df['count'].values
#     op_df['op_'+column+'_n'] = des_df['unique'].values
#     op_df['op_'+column+'_fre'] = des_df['freq'].values
    
    op_df['op_'+column+'_count'] = train_op.groupby('user')[column].count().values
    op_df['op_'+column+'_count'] = train_op.groupby('user')[column].nunique().values
    

op_type
op_mode
op_device
ip
net_type
channel
ip_3


In [744]:
from datetime import timedelta
def timedelta2sec(delta):
    day = delta.split('days')[0].strip()
    h,m,s = delta.split('days')[1].strip().split(':')
    sec=timedelta(days=int(day),hours=int(h),minutes=int(m),seconds=float(s)).total_seconds()
    return sec

train_op['time_diff_sec'] = train_op['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    op_df['op_time_'+operate]=train_op.groupby('user')['time_diff_sec'].agg(operate).values

In [745]:
op_df.fillna(0,inplace=True)

In [746]:
op_df.head()

,user,op_op_type_count,op_op_mode_count,op_op_device_count,op_ip_count,op_net_type_count,op_channel_count,op_ip_3_count,op_time_max,op_time_min,op_time_mean,op_time_median,op_time_std
0,Train_00000,11,12,2,3,3,4,3,683286.0,219874.0,3.058014e+05,263361.0,125051.535485
1,Train_00001,3,3,1,2,2,3,2,1263768.0,409809.0,6.598048e+05,438553.5,385614.886428
2,Train_00002,3,3,1,1,2,3,1,1283043.0,1282741.0,1.282845e+06,1282751.5,139.053690
3,Train_00003,8,8,2,18,2,3,5,1278388.0,199014.0,9.408690e+05,957412.0,247893.797669
4,Train_00004,2,2,1,1,1,2,1,631021.0,630973.0,6.309866e+05,630975.0,20.659138


In [747]:
op_df.shape

(41892, 13)

## 交易处理

In [472]:
train_trans.head()

,user,platform,tunnel_in,tunnel_out,amount,type1,ip,type2,ip_3,tm_diff
0,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,185784,45a1168437c708ff,NaN,11a213398ee0c623,NaN,19 days 09:02:45.000000000
1,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,391769,45a1168437c708ff,NaN,11a213398ee0c623,NaN,19 days 09:03:58.000000000
2,Train_08351,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,27740,674e8d5860bc033d,f10a09fe9e522a47,11a213398ee0c623,ee386d6f9fe45d0d,18 days 11:06:49.000000000
3,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,36689,f67d4b5a05a1352a,NaN,NaN,NaN,26 days 09:52:51.000000000
4,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,30746,f67d4b5a05a1352a,NaN,NaN,NaN,26 days 07:50:05.000000000


In [473]:
trans_df = pd.DataFrame()
group = train_trans.groupby(['user']).count()
trans_df['user'] = group.index

### 离散列，全部统计有几个类别

In [474]:
count_column = list(train_trans.columns[1:4])+list(train_trans.columns[5:9]) 
for column in count_column:
    print(column)
    trans_df['trans_'+column+'_count'] = train_trans.groupby('user')[column].count().values
    trans_df['trans_'+column+'_count'] = train_trans.groupby('user')[column].nunique().values
#     des_df = train_trans.groupby('user')[column].describe()
#     trans_df['trans_'+column+'_count'] = des_df['count'].values
#     trans_df['trans_'+column+'_n'] = des_df['unique'].values
#     trans_df['trans_'+column+'_fre'] = des_df['freq'].values

platform
tunnel_in
tunnel_out
type1
ip
type2
ip_3


### 处理amount

In [475]:
for operate in ['max','min','mean','median','std']:
    trans_df['trans_amount_'+operate]=train_trans.groupby(['user'])['amount'].agg(operate).values

### 处理时间

In [476]:
train_trans['time_diff_sec'] = train_trans['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    trans_df['trans_time_'+operate]=train_trans.groupby('user')['time_diff_sec'].agg(operate).values

### 标准差、freq有空值，用0填充

In [477]:
trans_df.fillna(0,inplace=True)

In [478]:
trans_df.head()

,user,op_platform_count,op_tunnel_in_count,op_tunnel_out_count,op_type1_count,op_ip_count,op_type2_count,op_ip_3_count,trans_amount_max,trans_amount_min,trans_amount_mean,trans_amount_median,trans_amount_std,trans_time_max,trans_time_min,trans_time_mean,trans_time_median,trans_time_std
0,Train_00000,2,1,1,4,2,1,2,267542,24798,53330.307692,30746.0,65274.138488,2478067.0,1169773.0,1.553383e+06,1627246.0,3.970320e+05
1,Train_00001,2,1,1,2,1,1,1,36098,36098,36098.000000,36098.0,0.000000,1821027.0,289554.0,1.055290e+06,1055290.5,1.082915e+06
2,Train_00002,2,1,1,3,3,1,3,162423,24980,57329.583333,34541.5,49227.699521,2665430.0,575019.0,1.454428e+06,625491.0,1.069315e+06
3,Train_00003,2,1,1,5,2,1,2,188880,25315,61652.454545,36689.0,62929.509668,2563665.0,830115.0,2.243363e+06,2562619.0,6.976849e+05
4,Train_00004,1,0,1,1,0,0,0,36689,36689,36689.000000,36689.0,0.000000,2014022.0,2014022.0,2.014022e+06,2014022.0,0.000000e+00


In [735]:
trans_df.shape

(41560, 18)

## 数据合并

In [784]:
# train_df = pd.merge(df_base, train_label, on='user', how='left')
train_df = pd.merge(df_base, op_df, on='user', how='left')


In [788]:
train_df.shape

(47782, 106)

In [786]:
train_df = pd.merge(train_df, trans_df, on='user', how='left')

In [789]:
train= train_df

In [790]:
train.head()

,user,province,city,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,label,product7_success_cnt,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period1_avg,login_cnt_period2_avg,login_cnt_period_avg,using_time_avg,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,op_op_type_count,op_op_mode_count,op_op_device_count,op_ip_count_x,op_net_type_count,op_channel_count,op_ip_3_count_x,op_time_max,op_time_min,op_time_mean,op_time_median,op_time_std,op_platform_count,op_tunnel_in_count,op_tunnel_out_count,op_type1_count,op_ip_count_y,op_type2_count,op_ip_3_count_y,trans_amount_max,trans_amount_min,trans_amount_mean,trans_amount_median,trans_amount_std,trans_time_max,trans_time_min,trans_time_mean,trans_time_median,trans_time_std
0,Train_06800,0.0,0.0,1,1,1,1,1,1,0,1,1,1,0,0,1,0.235862,0.415094,0.057692,0.098361,0.000000,0.091346,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.004402,0.003142,0.010150,0.021058,0.078467,0.00000,0.0,0.0,0.000000,0.057692,0.225496,0.003810,0.004135,0.002801,0.003509,0.883054,0.000000,0.834355,0.834355,0.0,0.899806,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,15.0,17.0,1.0,1.0,1.0,3.0,1.0,1238195.0,983947.0,1.065893e+06,1077918.5,89914.677544,1.0,1.0,1.0,1.0,0.0,1.0,0.0,33728.0,30746.0,31551.250000,30865.5,1455.533665,2620604.0,2367781.0,2.494192e+06,2494192.5,145967.427107
1,Train_23487,0.0,0.0,19,19,1,1,20,5,0,1,1,1,0,0,1,0.260690,0.245283,0.125000,0.213115,0.000000,0.062500,0.027397,0.0,0.252033,0.0,0.0,0.0,0.001736,0.004759,0.004061,0.045406,0.012243,0.167883,0.04878,0.0,1.0,0.857143,0.125000,0.235239,0.004431,0.004076,0.003265,0.003695,0.783694,0.853963,0.762022,0.751909,0.0,0.810893,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1025699.0,1025674.0,1.025683e+06,1025677.0,13.650397,1.0,1.0,1.0,2.0,1.0,1.0,1.0,503457.0,380126.0,441791.500000,441791.5,87208.186431,980425.0,980181.0,9.803030e+05,980303.0,172.534055
2,Train_36880,0.0,0.0,7,5,14,11,1,5,0,1,1,1,0,0,1,0.202759,0.320755,0.125000,0.213115,0.000000,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.005498,0.019068,0.010166,0.123932,0.018119,0.322993,0.04878,0.0,1.0,0.857143,0.125000,0.270565,0.014887,0.017896,0.008655,0.013557,0.645769,0.851046,0.619739,0.609661,0.0,0.657486,0.000008,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,4.0,1.0,1.0,2.0,3.0,1.0,579758.0,579581.0,5.796529e+05,579643.0,61.180017,3.0,1.0,1.0,4.0,2.0,2.0,2.0,787672.0,24798.0,307374.111111,166658.0,288304.779934,1962149.0,123248.0,1.612149e+06,1962074.0,706810.862319
3,Train_35392,0.0,0.0,11,7,1,1,12,4,0,1,1,1,0,0,1,0.320000,0.018868,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.203252,0.0,0.0,0.0,0.005498,0.003347,0.003779,0.049145,0.012243,0.155109,0.04878,0.0,0.0,0.857143,0.057692,0.243241,0.003550,0.002696,0.003046,0.002860,0.775081,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,10.0,11.0,1.0,10.0,3.0,3.0,7.0,907905.0,61355.0,4.752282e+05,545643.0,310441.845118,2.0,1.0,1.0,3.0,5.0,1.0,4.0,39630.0,24743.0,27326.555556,25923.0,4026.466663,2290284.0,1443970.0,1.875648e+06,1928058.0,291556.617186
4,Train_35057,0.0,0.0,8,9,1,1,9,5,0,2,8,1,0,0,1,0.344828,0.415094,0.125000,0.098361,0.070588,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.001736,0.003646,0.001435,0.032585,0.00293

In [750]:
# def label_smoothing(inputs, epsilon=0.1):
#     K = inputs.shape[-1]
#     return ((1-epsilon) * inputs) + (epsilon / K)

In [791]:
length = len(train)
train_length = int(0.90*length)

x = train.drop(columns=['user','label'],axis=1)[0:train_length]
y = train['label'].values[0:train_length]
# y = label_smoothing(y)
# 
valid_x = train.drop(columns=['user','label'],axis=1)[train_length:]
valid_y = train['label'].values[train_length:]
# valid_y = label_smoothing(valid_y)

In [792]:
x.shape

(43003, 104)

In [793]:
x = x.fillna(0)
valid_x = valid_x.fillna(0)

In [794]:
x.head()

,province,city,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,product7_success_cnt,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period1_avg,login_cnt_period2_avg,login_cnt_period_avg,using_time_avg,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,op_op_type_count,op_op_mode_count,op_op_device_count,op_ip_count_x,op_net_type_count,op_channel_count,op_ip_3_count_x,op_time_max,op_time_min,op_time_mean,op_time_median,op_time_std,op_platform_count,op_tunnel_in_count,op_tunnel_out_count,op_type1_count,op_ip_count_y,op_type2_count,op_ip_3_count_y,trans_amount_max,trans_amount_min,trans_amount_mean,trans_amount_median,trans_amount_std,trans_time_max,trans_time_min,trans_time_mean,trans_time_median,trans_time_std
0,0.0,0.0,1,1,1,1,1,1,0,1,1,1,0,0,1,0.235862,0.415094,0.057692,0.098361,0.000000,0.091346,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.004402,0.003142,0.010150,0.021058,0.078467,0.00000,0.0,0.000000,0.057692,0.225496,0.003810,0.004135,0.002801,0.003509,0.883054,0.000000,0.834355,0.834355,0.0,0.899806,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,15.0,17.0,1.0,1.0,1.0,3.0,1.0,1238195.0,983947.0,1.065893e+06,1077918.5,89914.677544,1.0,1.0,1.0,1.0,0.0,1.0,0.0,33728.0,30746.0,31551.250000,30865.5,1455.533665,2620604.0,2367781.0,2.494192e+06,2494192.5,145967.427107
1,0.0,0.0,19,19,1,1,20,5,0,1,1,1,0,0,1,0.260690,0.245283,0.125000,0.213115,0.000000,0.062500,0.027397,0.0,0.252033,0.0,0.0,0.0,0.001736,0.004759,0.004061,0.045406,0.012243,0.167883,0.04878,0.0,0.857143,0.125000,0.235239,0.004431,0.004076,0.003265,0.003695,0.783694,0.853963,0.762022,0.751909,0.0,0.810893,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1025699.0,1025674.0,1.025683e+06,1025677.0,13.650397,1.0,1.0,1.0,2.0,1.0,1.0,1.0,503457.0,380126.0,441791.500000,441791.5,87208.186431,980425.0,980181.0,9.803030e+05,980303.0,172.534055
2,0.0,0.0,7,5,14,11,1,5,0,1,1,1,0,0,1,0.202759,0.320755,0.125000,0.213115,0.000000,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.005498,0.019068,0.010166,0.123932,0.018119,0.322993,0.04878,0.0,0.857143,0.125000,0.270565,0.014887,0.017896,0.008655,0.013557,0.645769,0.851046,0.619739,0.609661,0.0,0.657486,0.000008,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,4.0,1.0,1.0,2.0,3.0,1.0,579758.0,579581.0,5.796529e+05,579643.0,61.180017,3.0,1.0,1.0,4.0,2.0,2.0,2.0,787672.0,24798.0,307374.111111,166658.0,288304.779934,1962149.0,123248.0,1.612149e+06,1962074.0,706810.862319
3,0.0,0.0,11,7,1,1,12,4,0,1,1,1,0,0,1,0.320000,0.018868,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.203252,0.0,0.0,0.0,0.005498,0.003347,0.003779,0.049145,0.012243,0.155109,0.04878,0.0,0.857143,0.057692,0.243241,0.003550,0.002696,0.003046,0.002860,0.775081,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,10.0,11.0,1.0,10.0,3.0,3.0,7.0,907905.0,61355.0,4.752282e+05,545643.0,310441.845118,2.0,1.0,1.0,3.0,5.0,1.0,4.0,39630.0,24743.0,27326.555556,25923.0,4026.466663,2290284.0,1443970.0,1.875648e+06,1928058.0,291556.617186
4,0.0,0.0,8,9,1,1,9,5,0,2,8,1,0,0,1,0.344828,0.415094,0.125000,0.098361,0.070588,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.001736,0.003646,0.001435,0.032585,0.002938,0.155109,0.04878,0.0,0.857143,0.120192,0.225519,0.002607,0.003001,0.000651,0.001898,0

In [804]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

# model = LogisticRegression()
# model.fit(x,y)

model = LGBMClassifier(n_estimators=1000, 
                       learning_rate=0.043,
                       subsample=0.8,
                       colsample_bytree=0.8,
                       reg_alpha=100,
                       reg_lambda=100)
model.fit(x, y,
          eval_set=(valid_x, valid_y),
          early_stopping_rounds=5
          )

prediction = model.predict_proba(valid_x)[:,1]
auc = roc_auc_score(valid_y,prediction)
auc

[1]	valid_0's binary_logloss: 0.562493
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.56103
[3]	valid_0's binary_logloss: 0.559613
[4]	valid_0's binary_logloss: 0.558271
[5]	valid_0's binary_logloss: 0.557083
[6]	valid_0's binary_logloss: 0.556032
[7]	valid_0's binary_logloss: 0.555043
[8]	valid_0's binary_logloss: 0.554091
[9]	valid_0's binary_logloss: 0.553244
[10]	valid_0's binary_logloss: 0.552316
[11]	valid_0's binary_logloss: 0.55148
[12]	valid_0's binary_logloss: 0.55067
[13]	valid_0's binary_logloss: 0.549953
[14]	valid_0's binary_logloss: 0.549313
[15]	valid_0's binary_logloss: 0.548692
[16]	valid_0's binary_logloss: 0.548158
[17]	valid_0's binary_logloss: 0.547537
[18]	valid_0's binary_logloss: 0.546947
[19]	valid_0's binary_logloss: 0.546488
[20]	valid_0's binary_logloss: 0.545979
[21]	valid_0's binary_logloss: 0.545475
[22]	valid_0's binary_logloss: 0.545043
[23]	valid_0's binary_logloss: 0.54463
[24]	valid_0's binary_logloss: 0.

[227]	valid_0's binary_logloss: 0.527118
[228]	valid_0's binary_logloss: 0.527118
[229]	valid_0's binary_logloss: 0.527108
[230]	valid_0's binary_logloss: 0.527137
[231]	valid_0's binary_logloss: 0.527098
[232]	valid_0's binary_logloss: 0.52709
[233]	valid_0's binary_logloss: 0.527061
[234]	valid_0's binary_logloss: 0.527031
[235]	valid_0's binary_logloss: 0.527047
[236]	valid_0's binary_logloss: 0.527015
[237]	valid_0's binary_logloss: 0.527033
[238]	valid_0's binary_logloss: 0.527016
[239]	valid_0's binary_logloss: 0.527007
[240]	valid_0's binary_logloss: 0.526959
[241]	valid_0's binary_logloss: 0.526921
[242]	valid_0's binary_logloss: 0.526884
[243]	valid_0's binary_logloss: 0.526904
[244]	valid_0's binary_logloss: 0.526884
[245]	valid_0's binary_logloss: 0.526872
[246]	valid_0's binary_logloss: 0.526887
[247]	valid_0's binary_logloss: 0.526856
[248]	valid_0's binary_logloss: 0.526863
[249]	valid_0's binary_logloss: 0.526855
[250]	valid_0's binary_logloss: 0.526868
[251]	valid_0's b

0.6808999258992909

In [796]:
joblib.dump(model,'model.pkl')

['model.pkl']

In [797]:
importance_df = pd.DataFrame({
        'column': x.columns,
        'importance': model.feature_importances_,
    }).sort_values(by='importance')[-100:]

In [798]:
importance_df

,column,importance
24,service1_cnt,0
48,service_amt_cnt_avg,0
52,3,0
53,4,0
54,5,0
58,9,0
13,product5_amount,0
12,product4_amount,0
57,8,0
61,12,0


In [ ]:
# select_feature=importance_df['column']

# x = x[select_feature]
# valid_x = valid_x[select_feature]

In [605]:
# from catboost import CatBoostClassifier


# model = CatBoostClassifier(
#     iterations=500,
#     random_seed=42,
#     logging_level='Silent'
# )

# model.fit(
#     x.values, y,
#     eval_set=(valid_x.values, valid_y),
# #     logging_level='Verbose',  # you can uncomment this for text output
#     plot=True
# )

# prediction = model.predict_proba(valid_x)[:,1]
# auc = roc_auc_score(valid_y,prediction)
# auc

baseline:
0.6769352059269422

添加trans amount的扩展变量：
0.6782946345739088

添加trans ip3的扩展变量：
0.6791192236191647

添加trans type直接数值化：
0.6796788342378327

添加所有的des特征，并修改了学习率：
0.6823851869318595

添加avg特征：
0.6831416556804824

In [ ]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 100)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)
